In [1]:
%matplotlib inline

import pandas
from sklearn.model_selection import train_test_split
import Recommender

# Load music data

In [3]:
#Load from server. It may take long for slow connections.
location = 'https://projects.danielvivas.com/song_recommender/fulldata.csv'

#Sample file for slow connections.
#location = '../data/processed/fulldata_sample.csv'

#Load from local
#location = '../data/processed/fulldata.csv'

song_df =  pandas.read_csv(location, index_col=0)

song_df

,SongNumber,AlbumID,AlbumName,ArtistID,ArtistName,Duration,Tempo,TimeSignature,Title,ArtistFamiliarity,ArtistHotttnesss,Loudness,StartOfFadeOut,UserID,playCount,song
SongID,,,,,,,,,,,,,,,,
SOCIWDW12A8C13D406,2,300822,Dimensions,ARMJAGH1187FB546F3,The Box Tops,148.03546,121.274,4,Soul Deep,0.630630,0.417500,-9.843,137.915,84ce6a9b05c928a12f052c78554ac74a88fb28b5,3,Soul Deep - The Box Tops
SOCIWDW12A8C13D406,2,300822,Dimensions,ARMJAGH1187FB546F3,The Box Tops,148.03546,121.274,4,Soul Deep,0.630630,0.417500,-9.843,137.915,6280dcb18e0a8d7e5b1e578b6e1d4edea1cfb9c8,1,Soul Deep - The Box Tops
SOCIWDW12A8C13D406,2,300822,Dimensions,ARMJAGH1187FB546F3,The Box Tops,148.03546,121.274,4,Soul Deep,0.630630,0.417500,-9.843,137.915,793a4e91553a4c61e516b66dc5bc9600a05ce1a3,1,Soul Deep - The Box Tops
SOCIWDW12A8C13D406,2,300822,Dimensions,ARMJAGH1187FB546F3,The Box Tops,148.03546,121.274,4,Soul Deep,0.630630,0.417500,-9.843,137.915,577304f2958b48b89e6acfa490e61ac5b643703b,2,Soul Deep - The Box Tops
SOCIWDW12A8C13D406,2,300822,Dimensions,ARMJAGH1187FB546F3,The Box Tops,148.03546,121.274,4,Soul Deep,0.630630,0.417500,-9.843,137.915,cd6f72af6795e918979be23251f5b14c642c9a00,1,Soul Deep - The Box Tops
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SOFAOMI12A6D4FA2D8,10000,64501,Once Upon A Shattered Life,ARYXOV81187B99831D,Seventh Day Slumber,209.73669,150.575,4,Shattered Life,0.609182,0.509243,-5.324,193.167,0e22f172b4023358903998b836a6fa392b2304e6,2,Shattered Life - Seventh Day Slumber
SOFAOMI12A6D4FA2D8,10000,64501,Once Upon A Shattered Life,ARYXOV81187B99831D,Seventh Day Slumber,209.73669,150.575,4,Shattered Life,0.609182,0.509243,-5.324,193.167,e593c4a4d6c5baa14a2dc69c39c54cd89bbc6a2c,1,Shattered Life - Seventh Day Slumber
SOFAOMI12A6D4FA2D8,10000,64501,Once Upon A Shattered Life,ARYXOV81187B99831D,Seventh Day Slumber,209.73669,150.575,4,Shattered Life,0.609182,0.509243,-5.324,193.167,ea762927a6f57267acdcca2b0d4b080b17ef85f9,4,Shattered Life - Seventh Day Slumber


# Explore data

Some useful stats to understand our dataset.

In [3]:
print('We have', len(song_df['UserID'].unique()), 'unique users and', len(song_df.index.unique()), 'unique songs.')
sum = song_df.groupby('UserID').sum('playCount').sort_values('playCount', ascending=False)

print('The user with most plays played', sum.iloc[0]['playCount'], 'times.')

print('Our dataset has', song_df.shape[0], 'records')

We have 417827 unique users and 3626 unique songs.
The user with most plays played 772.0 times.
Our dataset has 771510 records


### Get list of unique users and songs

In [4]:
users = song_df['UserID'].unique()
songs = song_df['song'].unique()

# Create a song recommender

First we divide our dataset in train and test data

In [5]:
train_data, test_data = train_test_split(song_df, test_size = 0.20, random_state=0)
print(train_data.head(5))

                    SongNumber  AlbumID                    AlbumName  \
SongID                                                                 
SOEHWGF12A6D4F8B2B         142   302176  Essential R&B Hit Selection   
SOSELMV12A6D4FCF5A        6018   229040                 ESSENTIAL 08   
SODQFQC12A8C13FD7A        8177   441073                The Open Door   
SOWEUOO12A6D4F6D0C        6024    85606          Minutes To Midnight   
SOLLOTO12AB01804C6        3773   532679                    Friend EP   

                              ArtistID                     ArtistName  \
SongID                                                                  
SOEHWGF12A6D4F8B2B  AR6PJ8R1187FB5AD70  Shakira Featuring Wyclef Jean   
SOSELMV12A6D4FCF5A  ARWD25M1187FB4C563                  Amy Winehouse   
SODQFQC12A8C13FD7A  ARVXU2X1187B9AE6D8                    Evanescence   
SOWEUOO12A6D4F6D0C  ARQUMH41187B9AF699                    Linkin Park   
SOLLOTO12AB01804C6  ARRDRI71187FB46EAA                   

In [6]:
def print_userItems(user_id):
    model = Recommender.recommender(train_data)

    user_items = model.get_user_items(user_id)

    print("------------------------------------------------------------------------------------")
    print("Training data songs for the user userid:", user_id)
    print("------------------------------------------------------------------------------------")

    for user_item in user_items:
        print(user_item)

    print("----------------------------------------------------------------------")

    return user_items



### Use the personalized model to make some song recommendations

In [9]:
#Create the model
model = Recommender.recommender(train_data)

#Select a random user
user_id = users[367]

#Get songs listened by this user
listened_songs = model.get_user_items(user_id)
print('---------------LISTENED SONGS-----------------')
for i in listened_songs:
    print(i)
print('----------------------------------------------\n')

#Get recommendations
rm = model.recommend(user_id)
rm

['Drop of Rain - Tweeterfriendly Music', 'Defend You (Album Version) - Silverstein', 'Bury Your Head - Saosin']
Non zero values in cooccurence_matrix :1045


,user_id,song,score,rank
0,f02301a1bc6035fc9889a17ac8bf89de3e039ca2,MY CONSOLATION (LP Version) - Silverstein,0.029841,1
1,f02301a1bc6035fc9889a17ac8bf89de3e039ca2,Meathead (Album Version) (Non-PA) - Story Of T...,0.019125,2
2,f02301a1bc6035fc9889a17ac8bf89de3e039ca2,Bury Your Head (Acoustic) - Saosin,0.017379,3
3,f02301a1bc6035fc9889a17ac8bf89de3e039ca2,"ll That's Left"" - Thrice",0.009074,4
4,f02301a1bc6035fc9889a17ac8bf89de3e039ca2,Mailbox Arson - Alexisonfire,0.007648,5
5,f02301a1bc6035fc9889a17ac8bf89de3e039ca2,Spin (Album Version) - Taking Back Sunday,0.007320,6
6,f02301a1bc6035fc9889a17ac8bf89de3e039ca2,Ability To Create A War - A Skylit Drive,0.006815,7
7,f02301a1bc6035fc9889a17ac8bf89de3e039ca2,I Am Fred Astaire (Album Version) - Taking Bac...,0.006586,8
8,f02301a1bc6035fc9889a17ac8bf89de3e039ca2,She Will Love You (Album Version) - Aiden,0.006441,9
9,f02301a1bc6035fc9889a17ac8bf89de3e039ca2,The Plot To Bomb The Panhandle (Album Version)...,0.006312,10


### Now let's use the model to find similar songs

In [11]:
song = 'Karma - Alicia Keys'

model.get_similar_items([song])

no. of unique songs in the training set: 3554
Non zero values in cooccurence_matrix :396


,user_id,song,score,rank
0,,Christmas In Jamaica - Toni Braxton Featuring ...,0.038494,1
1,,Only Human - Usher Featuring The Nu Beginning,0.026776,2
2,,Morning Glow - Michael Jackson,0.019499,3
3,,Anytime You Need A Friend - Mariah Carey,0.014805,4
4,,Over The Edge - Akon,0.014620,5
5,,Music Of The Sun - Rihanna,0.011690,6
6,,Good Girl Gone Bad - Rihanna,0.008645,7
7,,"f I Ain't Got You"" - Alicia Keys",0.008111,8
8,,Voodoo Doll - Fergie,0.007418,9
9,,"ey Daddy (Daddy's Home)"" - Usher",0.006550,10
